## Visualisieren einer FFT an einer periodischen Schwingung

**allgemeine harmonische Sinusschwingung:**<br><br>
$\quad {y(t)= \hat{y} ⋅ sin(\omega⋅t + \phi_0)}$<br> oder <br> $\quad {y(t)=\hat{y}⋅sin(2{\pi}f⋅t+φ_0)}$

$\hat{y} : Amplitude$ <br>
${\omega: Kreisfrequenz}$ in  ${1 \over s}$<br>
${f: Frequenz}$ in ${1 \over s}$;  Periodendauer Ts = ${1 \over f}$ in s<br>
${\phi}_0: Phasenwinkel$<br><br>


**Rechteckfunktion:**
$ x(t) = rect(at) = \begin{cases}
    1       & \quad \text{fürr |t|} \leq {1 \over 2a} \\
    0       & \quad \text{sonst }
  \end{cases}$
<br><br>
[wikipedia, Rechteckfunktion](https://de.wikipedia.org/wiki/Rechteckfunktion)


**Dreiecksfunktion:**
$ x(t) = tri(t) = \begin{cases}
    1-\text{|t|}       & \quad \text{für |t| < 1} \\
    0       & \quad \text{sonst}
  \end{cases}$
<br><br>
[wikipedia, Dreiecksfunktion](https://de.wikipedia.org/wiki/Dreiecksfunktion)

In [1]:
# resourcen
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

#set backend for interactive toolbar
%matplotlib widget

In [2]:
#presets
MAX_Amplitude = 5
MAX_Frequenz = 1000
MAX_Phase = 180

fNoise = 780
timeBase = np.arange(0, 3, 1/10000)     # zeitvektor für  0...3 s, sample rate:  1/fsample (default = 10kHz)

In [3]:
# funktionen definieren
#
def getSinus(amp,f,phase,time):
    return(amp*np.sin(2*np.pi*f*time + phase))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.square.html
def getRect(amp,f,time):
    return(amp*signal.square(2*np.pi*f*time))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.sawtooth.html#scipy.signal.sawtooth
def getTri(f,time):
    return(signal.sawtooth(2*np.pi*f*time, width=0.5))


def calc_FFT(fSample, signal):
    #https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft
    # fft mit numpy.fft - routinens
    #
    # DFT für signal berechnen
    fftSig = np.fft.fft(signal)
    #
    # anzahl der punkte des komplexen fft-vektors
    n = len(fftSig)
    # frequenzen den einzelnen Punkten zuordnen
    freq = np.fft.fftfreq(n, 1/fSample)
    #
    # positiven Bereich des komplexen Spektrums auswaehlen
    # die Nullfrequenzkomponente in die Mitte des Spektrums verschieben.
    # - amplituden und frequenzen
    Y1_shift = np.fft.fftshift(fftSig)
    F1_shift = np.fft.fftshift(freq)
    #
    # indes des Nullpunktes bestimmen
    #
    iZero = int(np.ceil(n/2.0))
    #
    # amplituden und frequenzen von 0 ... n/2 auswaehlen
    Y1_pos = Y1_shift[iZero:-1]
    F1_pos = F1_shift[iZero:-1]
    #
    # amplitude normalisieren mit (2* 1/n) und real-Anteil bestimmen
    #
    reSpectrum = 2 * 1/n * np.abs(Y1_pos)
    return(reSpectrum , F1_pos)
#

In [4]:
#interaktive Schalter
# 
# signal menu
#
signalForms = ['Sinus', 'Rechteck', 'Dreieck']
signal_dropdown = widgets.Dropdown(description='Signalform auswählen', options=signalForms, value='Sinus',\
                                   style={'description_width': 'initial'})
#Sliders
amp_1 = widgets.IntSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}_1$:")
frq_1 = widgets.IntSlider(min=0, max=MAX_Frequenz, value=10, step=5, description="$f_1$: in Hz")
phase_1 = widgets.IntSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_1: in °')
#
amp_2 = widgets.IntSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}_2$:")
frq_2 = widgets.IntSlider(min=0, max=MAX_Frequenz, value=10, step=5, description="$f_2$: in Hz")
phase_2 = widgets.IntSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_2: in °')
#
fSample = widgets.IntSlider(min=100, max=100000, value=10000, step=10, description='Abtastfrequenz',\
                            style={'description_width': 'initial'})
#
#checkbox for noise overlay
chkNoise = widgets.Checkbox(description='Störsignal hinzufügen')
#
# reset button um auf default-Werte zurückzu setzen
resetBtn = widgets.Button(description="Reset")

In [5]:
# Visualisierung
# Diagramme zeichnen 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
#
# 1. Callback-Funktion definieren mit der die Figuren aktualisiert werden, wenn sich ein Stueerelement geändert hat
def update_view(*args):
    
    # Zeitvektor aktualisieren
    timeBase = np.arange(0, 3, 1/fSample.value)     # zeitvektor für  0...3 s, sample rate:  1/fsample (default = 10kHz)
    
    #
    #Abtastfrequenz muss mindestens doppelt so hoch wie die höchste vorkommende Frequenz sein
    #(Nyquist - Theorem)
    # andernfalls: IndexError bei der FFT-Berechnung
    #
    if ((frq_1.value > 50) | (frq_2.value > 50)):
        fSample.min = 2* frq_1.value
        if (frq_2.value > frq_1.value):
            fSample.min = 2* frq_2.value
    
    if (chkNoise.value == True):        
        fSample.min = 2* np.max([frq_1.value, frq_2.value, fNoise])

    # dropdowns
    #
    signaltype = signal_dropdown.value
    
    #-----------------------------------------------------------
    # Ausgangssignal nach der gewählten Signalform rechnen
    #
    if (signaltype == "Sinus"):
        # evtl. ausgeschaltene Steuerlemente wieder aktivieren
        amp_2.disabled = False
        frq_2.disabled = False
        phase_2.disabled = False
        chkNoise.disabled = False
        phase_1.disabled = False

        ySin1 = getSinus(amp_1.value, frq_1.value, phase_1.value, timeBase)
        ySin2 = getSinus(amp_2.value, frq_2.value, phase_2.value, timeBase)
        ySum = ySin1 + ySin2
        if (chkNoise.value == True): # das Störsignal hat ein feste Frequenz von 780Hz und eine Amplitude von 5
            ySum = ySin1 + ySin2 + (5*np.sin(2*np.pi*fNoise*timeBase))
    
    if (signaltype == "Rechteck"):
        yRect1 = getRect(amp_1.value, frq_1.value, timeBase)
        # Steuerelemente des 2. (Sinus)-Signals deaktivieren
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        phase_1.disabled = True
        #
        ySum = yRect1
        #
    if (signaltype == "Dreieck"):
        yTri = getTri(frq_1.value, timeBase)
        # Steuerelemente des 2. (Sinus)-Signals deaktivieren        
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True        
        phase_1.disabled = True
        #
        ySum = yTri
        
    #-----------------------------------------------------------
    # FFT berechnen
    #
    # Amplitudenspektrum des original-Signals:
    #
    fftSig, frqSig = calc_FFT(fSample.value, ySum)
    #
    # Rücktransformation
    # (hier wieder die komplexe DFT verwenden)
    #
#     reOrig = np.fft.ifft(np.fft.fft(ySum))
    
    #-----------------------------------------------------------
    # Figuren aktualisiern
    #
    # angezeigten Zeitbereich auf 3 signalperioden begrenzen:
    if (frq_1.value < 2):
        xEnd = 3
    if (frq_1.value > 1):
        xEnd = 3* 1/frq_1.value
    if (signaltype == 'Sinus'): 
        if ((frq_1.value > 0) & (frq_2.value > 0) & (frq_1.value > frq_2.value)):
            xEnd = 3* 1/frq_2.value
    #
    # Frequenzbereich des FFT-diagrams begrenzen:
    fftError = True
    xFFTEnd = len(frqSig)
    add_f = 0
    peaks,_ = signal.find_peaks(fftSig, height=(0.01, 10))
    if (np.count_nonzero(peaks) > 0):
        fftError = False
        xFFTEnd = np.max(peaks)
        add_f = (len(frqSig) - xFFTEnd) -1
        if add_f > 100:
            add_f = 100
    
    # fig_1: original-signal im zeitbereich
    #
    axes[0].clear()
    axes[0].set_title("Ausgangssignal im Zeitbereich")
    axes[0].plot(timeBase, ySum, linestyle='-', color='r', label=r"$y(t)_{{{}}}$".format(signaltype))
    if (signaltype == "Sinus"):
        axes[0].plot(timeBase,ySin1, linewidth = 1, linestyle='--', color = 'b', label='$y_{1(t)}$')
        axes[0].plot(timeBase,ySin2, linewidth = 1, linestyle='dotted', color = '0.5', label='$y_{2(t)}$')
    axes[0].set_xlabel("Zeit /s")
    axes[0].set_ylabel("Amplitude")
    axes[0].set_xlim(timeBase[0], xEnd)
    axes[0].legend(loc="best")
    axes[0].grid(True)
    
    # fig_2: fft des original-signals
    #
    axes[1].clear()
    if (signaltype == "Sinus"):
        axes[1].set_title("FFT des Summensignals")
    else:
        axes[1].set_title("FFT des linken Signals")
    axes[1].plot(frqSig, fftSig, color='C1')
    axes[1].set_xlabel("f /Hz")
    axes[1].set_ylabel("Amplitude")
    if fftError != True:
        axes[1].set_xlim(frqSig[0], frqSig[xFFTEnd+add_f])
    else:
        yLim = plt.ylim()
        yPos = yLim[0] + yLim[-1] / 2
        bbox_props = dict(boxstyle="round", fc=(1, 0.4, 0.4), ec="0.5", alpha=0.9)
        axes[1].text(frqSig[int(np.ceil(len(frqSig)/2))], yPos, "Abtastfrequenz beachten",\
                     ha="center", va="center", size=10, bbox=bbox_props)

    axes[1].grid(True)
    
    fig.tight_layout()
    
def reset_controls(btn):
    # Reset Button:
    fSample.value = 10000
    signal_dropdown.value = 'Sinus'
    frq_1.value = 10
    frq_2.value = 10
    amp_1.value = 1
    amp_2.value = 1
    phase_1.value = 1
    phase_2.value = 1
    chkNoise.value = False
    update_view()
    

#--------------------------------------------------------
# 2. die Callback-Funktion mit der Funktion 'observe' den Steuerelementen zuweisen
signal_dropdown.observe(update_view, 'value')
fSample.observe(update_view,'value')
amp_1.observe(update_view,'value')
amp_2.observe(update_view,'value')
frq_1.observe(update_view,'value')
frq_2.observe(update_view,'value')
phase_1.observe(update_view,'value')
phase_2.observe(update_view,'value')
chkNoise.observe(update_view,'value')
resetBtn.on_click(reset_controls)

#--------------------------------------------------------
# Anwendung starten
#
# Diagramme einmal zeichnen
update_view()
#
#
# mit 'widgets.VBox / .HBox' die Steuerelemente arangieren
#
# box-layout definieren
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='flex-start',width='90%')

widgets.VBox([widgets.HBox([signal_dropdown, chkNoise]), fSample, widgets.HBox([amp_1, amp_2]),\
                            widgets.HBox([frq_1, frq_2]),widgets.HBox([phase_1, phase_2]),\
             resetBtn],\
            layout = box_layout)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Copyright © 2020 IUBH Internationale Hochschule